In [1]:
import os

import openai
from dotenv import load_dotenv

load_dotenv()

# APIキーを設定
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# LLMの準備
llm = ChatOpenAI(
    model="gpt-3.5-turbo", # モデル
    temperature=0, # ランダムさ
)

# 埋め込みモデルの準備
embeddings = OpenAIEmbeddings()

In [3]:
from langchain.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ドキュメントの読み込み
loader = DirectoryLoader("./data/")
documents = loader.load()

# ドキュメントの分割
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, # ドキュメントサイズ (トークン数)
    chunk_overlap=0, # 前後でオーバーラップするサイズ
    separators=["\n\n"] # セパレーター
).split_documents(documents)
print(len(documents))

14


In [4]:
from langchain_chroma import Chroma

# VectorStoreの準備
vectorstore = Chroma.from_documents(
    documents,
    embedding=embeddings,
)

In [5]:
# Retrieverの準備
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2},
)

In [6]:
from langchain_core.prompts import ChatPromptTemplate

# PromptTemplateの準備
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "次のコンテキストのみを使用して、この質問に答えてください。\n\n{context}"),
        ("human", "{input}"),
    ]
)

In [7]:
from langchain_core.runnables import RunnablePassthrough

# RAGチェーンの準備
rag_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt_template
    | llm
)

In [8]:
# 質問応答
response = rag_chain.invoke("ギターヒーローの正体は？")
print(response.content)

後藤ひとり（ごとう ひとり）


In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# コンテキストのフォーマット
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Generationチェーンの準備
gemeration_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt_template
    | llm
    | StrOutputParser()
)

# Retrieverチェーンの準備
retrieve_chain = (lambda x: x["input"]) | retriever

# ソース付きRAGチェーンの準備
rag_chain_with_source = RunnablePassthrough.assign(context=retrieve_chain).assign(
    answer=gemeration_chain
)

In [10]:
# 質問応答
rag_chain_with_source.invoke({"input": "ギターヒーローの正体は？"})

{'input': 'ギターヒーローの正体は？',
 'context': [Document(metadata={'source': 'data/bocchi.txt'}, page_content='後藤 ひとり (ごとう ひとり)：秀華高校に通う女子。桃色の髪を無造作に伸ばし、いつもジャージを身につけている。自他共に認める引きこもり一歩手前の「陰キャ」で、承認欲求が人一倍強いにもかかわらず、臆病な性格で人と接するのを極度に苦手としている。そのため、すぐに自分の世界に入って落ち込むという情緒不安定さを見せる。押し入れやダンボールに潜り込む癖があり、「完熟マンゴー仮面」を持ち歩いている。運動も勉強も苦手で、特に勉強はまじめに授業を受けているにもかかわらず、生来の要領の悪さから赤点ギリギリ。自分には何の取り柄もないのを痛感していたため、中学の頃に暗い性格の人間がバンドをやって人気者になったインタビューを読んで、ギターを始める。毎日練習したお陰でギターの腕前はプロ級になったが、結局、その腕前を披露する場に恵まれず、友達を作れないまま中学を卒業。現在は「ギターヒーロー」の名で動画配信している。ギタリストを探していた伊地知虹夏に誘われ、「結束バンド」に加入し、バンド活動を始める。バンド内ではギタリスト兼作詞を担当している。山田リョウに「ぼっち」の愛称を付けられて以降は、その愛称で呼ばれる。ギターの腕前はかなりのものだが、他人と合わせるセッションの経験が皆無なため、バンドではまともに演奏できないでいる。しかし徐々にバンドとして演奏できるようになり、文化祭ライブでは機材トラブルというアクシデントに見舞われながらも、アドリブでボトルネック奏法を行い、ライブを成功に導いた。なお、父親から借りたギターは文化祭で壊れたため、動画配信の広告収入で得たお金を使って、新たなギターを手に入れた。人見知りなため、美容院にもいけず、髪の毛を伸ばし放題で前髪で目をつねに隠している。それに加えて野暮ったいジャージ姿でいるため気づかれていないが、実は同性すら見とれるほどの美少女。黙って着飾っていれば「アイドル事務所に入れる」「ビジュアル担当」と言われるほどのポテンシャルを秘めているが、キメ顔は10秒しか持たず、ふだんの奇行がすべてを台無しにしている。'),
  Document(metadata={'

```json
{'input': 'ギターヒーローの正体は？',
 'context': [Document(metadata={'source': 'data/bocchi.txt'}, page_content='後藤 ひとり (ごとう ひとり)：秀華高校に通う女子。桃色の髪を無造作に伸ばし、いつもジャージを身につけている。自他共に認める引きこもり一歩手前の「陰キャ」で、承認欲求が人一倍強いにもかかわらず、臆病な性格で人と接するのを極度に苦手としている。そのため、すぐに自分の世界に入って落ち込むという情緒不安定さを見せる。押し入れやダンボールに潜り込む癖があり、「完熟マンゴー仮面」を持ち歩いている。運動も勉強も苦手で、特に勉強はまじめに授業を受けているにもかかわらず、生来の要領の悪さから赤点ギリギリ。自分には何の取り柄もないのを痛感していたため、中学の頃に暗い性格の人間がバンドをやって人気者になったインタビューを読んで、ギターを始める。毎日練習したお陰でギターの腕前はプロ級になったが、結局、その腕前を披露する場に恵まれず、友達を作れないまま中学を卒業。現在は「ギターヒーロー」の名で動画配信している。ギタリストを探していた伊地知虹夏に誘われ、「結束バンド」に加入し、バンド活動を始める。バンド内ではギタリスト兼作詞を担当している。山田リョウに「ぼっち」の愛称を付けられて以降は、その愛称で呼ばれる。ギターの腕前はかなりのものだが、他人と合わせるセッションの経験が皆無なため、バンドではまともに演奏できないでいる。しかし徐々にバンドとして演奏できるようになり、文化祭ライブでは機材トラブルというアクシデントに見舞われながらも、アドリブでボトルネック奏法を行い、ライブを成功に導いた。なお、父親から借りたギターは文化祭で壊れたため、動画配信の広告収入で得たお金を使って、新たなギターを手に入れた。人見知りなため、美容院にもいけず、髪の毛を伸ばし放題で前髪で目をつねに隠している。それに加えて野暮ったいジャージ姿でいるため気づかれていないが、実は同性すら見とれるほどの美少女。黙って着飾っていれば「アイドル事務所に入れる」「ビジュアル担当」と言われるほどのポテンシャルを秘めているが、キメ顔は10秒しか持たず、ふだんの奇行がすべてを台無しにしている。'),
  Document(metadata={'source': 'data/bocchi.txt'}, page_content='佐藤 愛子 (さとう あいこ)：フリーライターとして活動する女性。ペンネームは「ぽいずん♥やみ」。年齢は23歳だが、童顔で小柄な体型をしているため、中学生でも通る幼い容姿をしている。佐藤愛子自身も幼い容姿は自覚しており、「14歳」と年齢を詐称したこともある。黒い髪をツインアップテールにしている。痛ロリ系のファッションに身を包んでおり、自分でも自分の格好を「痛い奴」と認識している。いいバンドをもっと多くの人に知ってもらいたいとの思いから、音楽系の記事を扱うサイト「ばんらぼ」の運営会社に所属し、記事を書いている。しかし大手のバンドに取材するツテも、音楽雑誌の編集者に就職するツテもないため、最近はアクセス数稼ぎのための記事しか書いておらず辟易している。後藤ひとりの文化祭の奇行が話題となり、その取材のために結束バンドを取材する。その際に、彼女たちの演奏する姿を見て、ひとりがギターヒーローであることに気づき、それをバンドメンバーに暴露した。ギターヒーローのファンだが、結束バンドのバンド演奏は「ガチじゃない」と酷評し、ひとりにバンドを抜けて一人で活動するように勧めた。この行動が結束バンドのメンバーを奮起させ、フェスに参加させるきっかけをつくった。言葉は厳しいが、結束バンドへの評価は的を得ており、ふだんの言動は痛いが音楽に関することには真摯に向き合っている。その後も結束バンドのことは何かと気に掛けており、彼女たちが着々と実力を付けていっているのも認めている。\n\nひとりの父 (ひとりのちち)：後藤ひとりの父親。本名は不明で、素顔もなぜか毎回明かされていないために不明。若い頃、ギターを弾いていたらしく、ひとりの中学時代、ギターに興味を覚えた娘に自分のギターを貸した。動画配信のアカウントを家族で共有しているため、ひとりが「ギターヒーロー」として人気を集めていることも知っており、ひとりのために広告収入を貯金していた。のちに、そのお金を新しいギターを購入する資金として渡している。変な特技を数多く持っており、隠れ身の術で壁に隠れていたり、完熟マンゴー仮面を凝ったデザインに改造したりしている。後藤ふたりから家族ヒエラルキーの中で最下位に置かれており、なんとかそれを挽回したいと考えている。')],
 'answer': '後藤ひとり（ごとう ひとり）'}
 ```